In [4]:
import numpy as np
import pandas as pd

In [5]:
tags = pd.read_csv('ml-20m/genome-tags.csv')
scores = pd.read_csv('ml-20m/genome-scores.csv')
movies = pd.read_csv('ml-20m/movies.csv')

In [6]:
# f = open('Hybrid/feature_extraction/Genome/movie_genomes.npy')
A = np.load('ml-20m/movie_genomes.npy', allow_pickle=True)
Asortedidx = (-A).argsort(axis=-1)

In [7]:
rating_thres = 3.5
user_ratings = pd.read_csv('ml-20m/ratings.csv')
user_ratings_above_thres = user_ratings[user_ratings['rating'] > rating_thres]

In [74]:
user_ratings_above_thres[:100]

,userId,movieId,rating,timestamp
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826
10,1,293,4.0,1112484703
...,...,...,...,...
184,2,541,5.0,974821014
185,2,589,5.0,974820658
187,2,908,4.0,974820691
188,2,924,5.0,974821014


In [8]:
user_movie_dict = {}
# max_user_id = user_ratings.userId.max()
max_user_id = min(user_ratings_above_thres.userId.max(), 150000)
num_items_list = []

In [9]:
len(list(set(user_ratings_above_thres.userId.tolist())))

138287

In [ ]:
for uid in range(max_user_id):
    # if uid not in user_movie_dict.keys() :
    rating_list = user_ratings_above_thres[ user_ratings_above_thres['userId'] == uid ]
    num_items_list.append(len(rating_list))
    user_movie_dict[uid] = rating_list
    
np.save('rating_dict.npy', user_movie_dict)

In [44]:
user_movie_dict = np.load('rating_dict.npy', allow_pickle=True).item()

In [45]:
empty_keys = []
for key in user_movie_dict:
    if len(user_movie_dict[key]) == 0:
        empty_keys.append(key)

print(len(empty_keys))
for key in empty_keys:
    del user_movie_dict[key]

207


In [46]:
print(user_movie_dict[1])

[151, 223, 253, 260, 293, 296, 318, 541, 1036, 1079, 1090, 1097, 1196, 1198, 1200, 1214, 1215, 1219, 1240, 1249, 1258, 1259, 1266, 1278, 1321, 1333, 1358, 1374, 1387, 1967, 2021, 2100, 2118, 2138, 2140, 2143, 2173, 2174, 2193, 2288, 2291, 2542, 2628, 2762, 2872, 2944, 2959, 2968, 3081, 3153, 3479, 3489, 3499, 3889, 3996, 4011, 4027, 4128, 4306, 4467, 4571, 4754, 4896, 4911, 4993, 5026, 5039, 5171, 5540, 5797, 5816, 5952, 6093, 6333, 6539, 6754, 6774, 7046, 7153, 7389, 7438, 7454, 7757, 8368, 8507, 8636, 8961, 31696]


In [47]:
user_ids = user_movie_dict.keys()
movie_ids = list(set(user_ratings.movieId.tolist()))
movie_ids.insert(0, 0)   # Insert padding item
# movie_ids = [0, *movie_ids]

movie_ids_to_idx = {}
for i, mid in enumerate(movie_ids):
    movie_ids_to_idx[mid] = i
user_idxs = np.arange(len(user_ids))

num_movies = []
for key in user_movie_dict:
    num_movies.append(len(user_movie_dict[key]))

num_total_movies = len(movie_ids)

# Convert original movie idx into 
for uid in user_movie_dict:
    for idx in range(len(user_movie_dict[uid])):
        user_movie_dict[uid][idx] = movie_ids_to_idx[user_movie_dict[uid][idx]]

In [48]:
print(user_movie_dict[1])

[152, 223, 253, 260, 293, 296, 318, 540, 1020, 1060, 1071, 1078, 1174, 1176, 1178, 1191, 1192, 1196, 1215, 1224, 1233, 1234, 1241, 1253, 1295, 1307, 1331, 1346, 1359, 1886, 1940, 2019, 2037, 2057, 2059, 2062, 2092, 2093, 2112, 2206, 2209, 2460, 2546, 2679, 2789, 2861, 2876, 2885, 2997, 3069, 3393, 3402, 3412, 3799, 3905, 3920, 3936, 4037, 4215, 4376, 4480, 4662, 4804, 4819, 4901, 4934, 4947, 5079, 5447, 5702, 5721, 5857, 5998, 6238, 6433, 6648, 6668, 6938, 7045, 7281, 7316, 7332, 7459, 7773, 7864, 7957, 8282, 9821]


## Statistics
- num. of total users : 138,286
- num. of movies per users : 72.27 +- 102.24 (max=3177, min=1)
- num. of unique movies : 


In [54]:
def pad_list(input_list, padded_len=128):
    padding = []
    input_list = list(input_list)
    if len(input_list) < padded_len:
        padding = [0] * (padded_len - len(input_list))
    output_list = input_list + padding
    output_list = output_list[:padded_len]
    return output_list

In [55]:
def generate_masked_binary_vector(input_list, mask_list):    
    bin_vector = np.zeros(num_total_movies, dtype=float)
    bin_vector[np.array(input_list, dtype=int)] = 1.
    bin_vector[mask_list] = 0.
    return bin_vector
    

In [69]:
from transformer import TransformerModel
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_batch(batch_size=20, input_size=35):
    # choose num_users
    user_idx = np.random.choice(user_idxs, batch_size)
    np.random.shuffle(user_idx)
    sources = []
    targets = []
    
    # choose movie_seen and movie_to_be_seen (padding if input_size > num_movie_seen)
    for uid in user_idx:
        # original_bivec = generate_masked_binary_vector(user_movie_dict[uid], [])

        source_mask_idx = np.random.choice(np.arange(num_total_movies), num_total_movies//2, replace=False)
        target_mask_idx = np.arange(num_total_movies)[~np.isin(np.arange(num_total_movies), source_mask_idx)]

        source_bivec = generate_masked_binary_vector(user_movie_dict[uid], source_mask_idx)
        target_bivec = generate_masked_binary_vector(user_movie_dict[uid], target_mask_idx)
        
        source = source_bivec.nonzero()[0]
        source = pad_list(list(source))
        source = np.array(source)
        
        sources.append(source)
        targets.append(target_bivec)

    sources = torch.Tensor(sources).to(device)
    targets = torch.Tensor(targets).to(device)

    return sources, targets
    

In [70]:
data, targets = get_batch()

In [71]:
targets.shape

torch.Size([20, 26745])

In [72]:
data.shape

torch.Size([20, 128])

In [29]:
nitems = num_total_movies
emsize = 200
nhid = 200
nlayers = 2
nhead = 2
dropout = 0.2
input_size = 35
model = TransformerModel(nitems, emsize, nhead, nhid, nlayers, dropout, use_posenc=False).to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
lr = 5.0 # 학습률
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # 학습 모드를 시작합니다.
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, input_size - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # 평가 모드를 시작합니다.
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)